In [ ]:
def check_anomaly(row):
    if (row['timestamp'].hour < 4 and row['count'] > 40):
        return 1
    elif (row['count'] > 60):
        return 1
    elif (row['difference'] > 30):
        return 1
    elif (row['count'] < 10 and row['timestamp'].hour >= 15 and row['timestamp'].hour <=19):
        return 1
    else:
        return 0

# Apply the function to create the 'actualAnomaly' column
df_combined['actualAnomaly'] = df_combined.apply(check_anomaly, axis=1)

df_combined['pastAnomaly'] =  df_combined['actualAnomaly'].shift(1)
df_combined['nextAnomaly'] =  df_combined['actualAnomaly'].shift(-1)

df_combined['pastAnomaly'] = df_combined['pastAnomaly'].fillna(0)
df_combined['nextAnomaly'] = df_combined['nextAnomaly'].fillna(0)

df_combined['ultraAnomaly'] = df_combined.apply(lambda x: max(x['actualAnomaly'], x['pastAnomaly'] *  x['nextAnomaly']),axis=1) 

df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_train['actualAnomaly'] = df_train.apply(check_anomaly, axis=1)
df_test['actualAnomaly'] = df_test.apply(check_anomaly, axis=1)

anomaly = df_combined.loc[df_combined['ultraAnomaly'] == 1, ['timestamp', 'count']]
print(anomaly.shape)
plt.figure(figsize=(20, 6))
plt.plot(df_combined['timestamp'], df_combined['count'], label='Time Series')
plt.scatter(anomaly['timestamp'],anomaly['count'], color='red', label='Detected Outliers')
plt.xlabel('minute')
plt.ylabel('count')
plt.legend()
plt.show()